<a href="https://colab.research.google.com/github/tienhuynh96/End-to-end-Question-Answering/blob/main/%5BDemo%5DReader_DistilBERT_QA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **1. Install and import bibraries**

In [ ]:
# Install library, -qq is quiet mode
# The datasets library by Hugging Face is used for accessing and managing machine learning datasets.
# The evaluate library used for evaluation metrics.
# The transformer library used for working with transformer models like BERT, GPT, etc
!pip install -qq datasets==2.16.1 evaluate==0.4.1 transformers[sentencepiece]==4.35.2
# Accelerate library is used to optimize and distribute PyTorch models and training
!pip install -qq accelerate==0.26.1
# Git Large File Storage (Git LFS) is an extension for Git that allows to manage large files (such as model weights and datasets) with Git.
!apt install git-lfs

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 73.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 15.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 13.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 37.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.9/270.9 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 43.6 MB/s eta 0:00:00
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
git-lfs is already the newest version (3.0.2-1ubuntu0.2).
0 upgraded, 0 newly installed,

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
import numpy as np
# Use tqdm for tracking the progress bar
from tqdm.auto import tqdm
import collections

import torch

from datasets import load_dataset
from transformers import AutoTokenizer                  # Tokenizer
from transformers import AutoModelForQuestionAnswering  # Pretrained model for QA
from transformers import TrainingArguments
from transformers import Trainer
import evaluate

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


## **2. Setup config**

In [ ]:
# Sử dụng mô hình "distilbert-base-uncased" làm mô hình checkpoint
MODEL_NAME =  "distilbert-base-uncased"

# Chiều dài tối đa cho mỗi đoạn văn bản sau khi được xử lý
MAX_LENGTH = 384

# Khoảng cách giữa các điểm bắt đầu của các đoạn văn bản liên tiếp
# Dùng để tách các context dài
STRIDE = 128

## **3. Setup Dataset**

### **3.1. Download dataset**

In [ ]:
# Download squad dataset từ HuggingFace
DATASET_NAME = 'squad_v2'
raw_datasets = load_dataset(DATASET_NAME)

### **3.2. EDA dataset**

In [ ]:
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 130319
    })
    validation: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 11873
    })
})

In [ ]:
# Print các thông tin Context, Question, Answer trong dataset
print("Context: ", raw_datasets["train"][0]["context"])
print("Question: ", raw_datasets["train"][0]["question"])
print("Answer: ", raw_datasets["train"][0]["answers"])

Context:  Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ bee-YON-say) (born September 4, 1981) is an American singer, songwriter, record producer and actress. Born and raised in Houston, Texas, she performed in various singing and dancing competitions as a child, and rose to fame in the late 1990s as lead singer of R&B girl-group Destiny's Child. Managed by her father, Mathew Knowles, the group became one of the world's best-selling girl groups of all time. Their hiatus saw the release of Beyoncé's debut album, Dangerously in Love (2003), which established her as a solo artist worldwide, earned five Grammy Awards and featured the Billboard Hot 100 number-one singles "Crazy in Love" and "Baby Boy".
Question:  When did Beyonce start becoming popular?
Answer:  {'text': ['in the late 1990s'], 'answer_start': [269]}


In [ ]:
# Check max number of answers
answer_num = [len(x["text"]) for x in raw_datasets["train"]["answers"]]
print("max number answer")
print(max(answer_num))
# Check len of context
context_len = [len(x.split()) for x in raw_datasets["train"]["context"]]
max_len_context = max(context_len)
print("maxt len context")
context_len.index(max_len_context)

max number answer
1
maxt len context


81510

In [ ]:
# Filter a dataset to exclude entries where there are no answers
non_answers = raw_datasets["train"].filter(
    lambda x: len(x['answers']['text']) > 0    # Answers present
)

Filter:   0%|          | 0/130319 [00:00<?, ? examples/s]

In [ ]:
non_answers[0]

{'id': '56be85543aeaaa14008c9063',
 'title': 'Beyoncé',
 'context': 'Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ bee-YON-say) (born September 4, 1981) is an American singer, songwriter, record producer and actress. Born and raised in Houston, Texas, she performed in various singing and dancing competitions as a child, and rose to fame in the late 1990s as lead singer of R&B girl-group Destiny\'s Child. Managed by her father, Mathew Knowles, the group became one of the world\'s best-selling girl groups of all time. Their hiatus saw the release of Beyoncé\'s debut album, Dangerously in Love (2003), which established her as a solo artist worldwide, earned five Grammy Awards and featured the Billboard Hot 100 number-one singles "Crazy in Love" and "Baby Boy".',
 'question': 'When did Beyonce start becoming popular?',
 'answers': {'text': ['in the late 1990s'], 'answer_start': [269]}}

In [ ]:
len(non_answers)

86821

### **3.3. Load tokenizer and run some examples**

In [ ]:
# Load tokenizer để run một số examples
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [ ]:
# Lấy ra 1 example từ tập train
context = raw_datasets["train"][81510]["context"]
question = raw_datasets["train"][81510]["question"]


In [ ]:
print(context)
print(question)

The sky is usually clear above the desert and the sunshine duration is extremely high everywhere in the Sahara. Most of the desert enjoys more than 3,600 h of bright sunshine annually or over 82% of the time and a wide area in the eastern part experiences in excess of 4,000 h of bright sunshine a year or over 91% of the time, and the highest values are very close to the theoretical maximum value. A value of 4,300 h or 98% of the time would be recorded in Upper Egypt (Aswan, Luxor) and in the Nubian Desert (Wadi Halfa). The annual average direct solar irradiation is around 2,800 kWh/(m2 year) in the Great Desert. The Sahara has a huge potential for solar energy production. The constantly high position of the sun, the extremely low relative humidity, the lack of vegetation and rainfall make the Great Desert the hottest continuously large area worldwide and certainly the hottest place on Earth during summertime in some spots. The average high temperature exceeds 38 °C (100.4 °F) - 40 °C (

In [ ]:
# Sử dụng tokenizer để mã hóa đữ liệu đầu hòa
inputs = tokenizer(
    question,     # Danh sách các question
    context,      # Danh sách các context
    max_length = MAX_LENGTH,   # Độ dài tối đa cho đầu ra mã hóa
    truncation="only_second",  # Cắt bớt dữ liệu chỉ cho phần thứ hai (context)
    stride=STRIDE,             # Độ dài bước nhảy trong trường hợp dữ liệu dài hơn max_length
    return_overflowing_tokens=True,  # Trả về các tokens vượt quá độ dài tối đa
    return_offsets_mapping=True,     # Trả về bản đồ vị trí của các tokens trong văn bản gốc
    padding="max_length"       # Điền vào dữ liệu để có cùng độ dài max_length
)

In [ ]:
inputs.keys()

dict_keys(['input_ids', 'attention_mask', 'offset_mapping', 'overflow_to_sample_mapping'])

In [ ]:
inputs

{'input_ids': [[101, 2054, 2003, 1996, 2922, 20930, 10843, 2312, 2181, 4969, 1029, 102, 1996, 3712, 2003, 2788, 3154, 2682, 1996, 5532, 1998, 1996, 9609, 9367, 2003, 5186, 2152, 7249, 1999, 1996, 19604, 1012, 2087, 1997, 1996, 5532, 15646, 2062, 2084, 1017, 1010, 5174, 1044, 1997, 4408, 9609, 6604, 2030, 2058, 6445, 1003, 1997, 1996, 2051, 1998, 1037, 2898, 2181, 1999, 1996, 2789, 2112, 6322, 1999, 9987, 1997, 1018, 1010, 2199, 1044, 1997, 4408, 9609, 1037, 2095, 2030, 2058, 6205, 1003, 1997, 1996, 2051, 1010, 1998, 1996, 3284, 5300, 2024, 2200, 2485, 2000, 1996, 9373, 4555, 3643, 1012, 1037, 3643, 1997, 1018, 1010, 3998, 1044, 2030, 5818, 1003, 1997, 1996, 2051, 2052, 2022, 2680, 1999, 3356, 5279, 1006, 2004, 7447, 1010, 28359, 2953, 1007, 1998, 1999, 1996, 16371, 15599, 5532, 1006, 28380, 2431, 2050, 1007, 1012, 1996, 3296, 2779, 3622, 5943, 20868, 12173, 18963, 2003, 2105, 1016, 1010, 5385, 6448, 2232, 1013, 1006, 25525, 2095, 1007, 1999, 1996, 2307, 5532, 1012, 1996, 19604, 2038, 1

In [ ]:
tokenizer.decode(inputs['input_ids'][1])

"[CLS] what is the largest hottest continuously large area worldwide? [SEP] high temperature was 47 °c ( 116. 6 °f ) in a remote desert town in the algerian desert called bou bernous with an elevation of 378 meters above sea level. it's the world's highest recorded average high temperature and only death valley, california rivals it. other hot spots in algeria such as adrar, timimoun, in salah, ouallene, aoulef, reggane with an elevation between 200 and 400 meters above sea level get slightly lower summer average highs around 46 °c ( 114. 8 °f ) during the hottest months of the year. salah, well known in algeria for its extreme heat, has an average high temperature of 43. 8 °c ( 110. 8 °f ), 46. 4 °c ( 115. 5 °f ), 45. 5 ( 113. 9 °f ). furthermore, 41. 9 °c ( 107. 4 °f ) in june, july, august and september. in fact, there are even hotter spots in the sahara, but they are located in extremely remote areas, especially in the azalai, lying in northern mali. the major part of the desert ex

In [ ]:
inputs['overflow_to_sample_mapping']

[0, 0, 0]

## **4. Tokenize dataset**

### **4.1. Tokenize train set**

In [ ]:
# Định nghĩa hàm preprocess_training_examples và nhận đối số examples là dữ liệu đào tạo
def preprocess_training_examples(examples):
    # Trích xuất danh sách câu hỏi từ examples và loại bỏ các khoảng trắng dư thừa
    questions = [q.strip() for q in examples["question"]]

    # Tiến hành mã hóa thông tin đầu vào sử dụng tokenizer
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=MAX_LENGTH,
        truncation="only_second",
        stride=STRIDE,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    # Trích xuất offset_mapping từ inputs và loại bỏ nó ra khỏi inputs
    offset_mapping = inputs.pop("offset_mapping")

    # Trích xuất sample_map từ inputs và loại bỏ nó ra khỏi inputs
    sample_map = inputs.pop("overflow_to_sample_mapping")

    # Trích xuất thông tin về câu trả lời (answers) từ examples
    answers = examples["answers"]

    # Khởi tạo danh sách các vị trí bắt đầu và kết thúc câu trả lời
    start_positions = []
    end_positions = []

    # Duyệt qua danh sách offset_mapping
    for i, offset in enumerate(offset_mapping):
        # Xác định index của mẫu (sample) liên quan đến offset hiện tại
        sample_idx = sample_map[i]

        # Trích xuất sequence_ids từ inputs (or segment)
        # Sequence_ids helps to distinguish between the question and the answer within the tokenized input sequence
        sequence_ids = inputs.sequence_ids(i)

        # Xác định vị trí bắt đầu và kết thúc của ngữ cảnh
        idx = 0
        while sequence_ids[idx] != 1:
            idx += 1
        context_start = idx
        while sequence_ids[idx] == 1:
            idx += 1
        context_end = idx - 1

        # Trích xuất thông tin về câu trả lời cho mẫu này
        answer = answers[sample_idx]

        if len(answer['text']) == 0:
            start_positions.append(0)
            end_positions.append(0)
        else:
            # Xác định vị trí ký tự bắt đầu và kết thúc của câu trả lời trong ngữ cảnh
            start_char = answer["answer_start"][0]
            end_char = answer["answer_start"][0] + len(answer["text"][0])

            # Nếu câu trả lời không nằm hoàn toàn trong ngữ cảnh, gán nhãn là (0, 0)
            if offset[context_start][0] > start_char or offset[context_end][1] < end_char:
                start_positions.append(0)
                end_positions.append(0)
            else:
                # Nếu không, gán vị trí bắt đầu và kết thúc dựa trên vị trí của các mã thông tin
                idx = context_start
                while idx <= context_end and offset[idx][0] <= start_char:
                    idx += 1
                start_positions.append(idx - 1)

                idx = context_end
                while idx >= context_start and offset[idx][1] >= end_char:
                    idx -= 1
                end_positions.append(idx + 1)

    # Thêm thông tin vị trí bắt đầu và kết thúc vào inputs
    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions

    return inputs


In [ ]:
# Tạo một biến train_dataset và gán cho nó giá trị sau khi áp dụng hàm preprocess_training_examples lên tập dữ liệu "train"
# Bật chế độ xử lý theo từng batch bằng cách đặt batched=True
# Loại bỏ các cột không cần thiết trong tập dữ liệu "train" bằng cách sử dụng remove_columns

train_dataset = raw_datasets["train"].map(
    preprocess_training_examples,
    batched=True,
    remove_columns=raw_datasets["train"].column_names,
)

# In ra độ dài của tập dữ liệu "train" ban đầu và độ dài của tập dữ liệu đã được xử lý (train_dataset)
len(raw_datasets["train"]), len(train_dataset)

Map:   0%|          | 0/130319 [00:00<?, ? examples/s]

(130319, 131754)

### **4.2. Tokenize Val set**

In [ ]:
def preprocess_validation_examples(examples):
    # Chuẩn bị danh sách câu hỏi bằng cách loại bỏ khoảng trắng ở đầu và cuối mỗi câu hỏi
    questions = [q.strip() for q in examples["question"]]

    # Sử dụng tokenizer để mã hóa các câu hỏi và văn bản liên quan
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=MAX_LENGTH,
        truncation="only_second",
        stride=STRIDE,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    # Lấy ánh xạ để ánh xạ lại ví dụ tham chiếu cho từng dòng trong inputs
    sample_map = inputs.pop("overflow_to_sample_mapping")
    example_ids = []

    # Xác định ví dụ tham chiếu cho mỗi dòng đầu vào và điều chỉnh ánh xạ offset
    for i in range(len(inputs["input_ids"])):
        sample_idx = sample_map[i]
        example_ids.append(examples["id"][sample_idx])

        sequence_ids = inputs.sequence_ids(i)
        offset = inputs["offset_mapping"][i]

        # Loại bỏ các offset không phù hợp với sequence_ids
        inputs["offset_mapping"][i] = [
            o if sequence_ids[k] == 1 else None for k, o in enumerate(offset)
        ]

    # Thêm thông tin ví dụ tham chiếu vào đầu vào
    inputs["example_id"] = example_ids

    return inputs

In [ ]:
# Tạo một biến validation_dataset và gán giá trị bằng việc sử dụng dữ liệu từ raw_datasets["validation"] sau khi áp dụng một hàm xử lý trước.
validation_dataset = raw_datasets["validation"].map(
    preprocess_validation_examples,  # Gọi hàm preprocess_validation_examples để xử lý dữ liệu đầu vào.
    batched=True,  # Xử lý dữ liệu theo từng batch.
    remove_columns=raw_datasets["validation"].column_names,  # Loại bỏ các cột không cần thiết từ dữ liệu ban đầu.
)

# In ra độ dài của raw_datasets["validation"] và validation_dataset để so sánh.
len(raw_datasets["validation"]), len(validation_dataset)

Map:   0%|          | 0/11873 [00:00<?, ? examples/s]

(11873, 12134)

## **5. Train model**

In [ ]:
# Load model
model = AutoModelForQuestionAnswering.from_pretrained(MODEL_NAME)

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForQuestionAnswering were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# Tạo đối tượng args là các tham số cho quá trình huấn luyện
args = TrainingArguments(
    output_dir="distilbert-finetuned-squadv2",   # Thư mục lưu trữ kết quả huấn luyện
    evaluation_strategy="no", # Chế độ đánh giá không tự động sau mỗi epoch
    save_strategy="epoch",    # Lưu checkpoint sau mỗi epoch
    learning_rate=2e-5,       # Tốc độ học
    num_train_epochs=2,       # Số epoch huấn luyện
    weight_decay=0.01,        # Giảm trọng lượng mô hình để tránh overfitting
    fp16=True,           # Sử dụng kiểu dữ liệu half-precision (16-bit) để tối ưu hóa tài nguyên
    push_to_hub=True,    # Đẩy kết quả huấn luyện lên một nơi chia sẽ trực tuyến (Hub)

)

In [ ]:
# Khởi tạo một đối tượng trainer để huấn luyện mô hình.
trainer = Trainer(
    model=model,          # Sử dụng mô hình đã được tạo trước đó
    args=args,            # Các tham số và cấu hình huấn luyện
    train_dataset=train_dataset,      # Sử dụng tập dữ liệu huấn luyện
    eval_dataset=validation_dataset,  # Sử dụng tập dữ liệu đánh giá
    tokenizer=tokenizer,  # Sử dụng tokenizer để xử lý văn bản
)

# Bắt đầu quá trình huấn luyện
trainer.train()

You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
500,3.205000
1000,2.320600
1500,2.070500
2000,1.871300
2500,1.737400
3000,1.663400
3500,1.595700
4000,1.589200
4500,1.526300
5000,1.496600


TrainOutput(global_step=32940, training_loss=1.2067382006480176, metrics={'train_runtime': 4037.392, 'train_samples_per_second': 65.267, 'train_steps_per_second': 8.159, 'total_flos': 2.582110281242419e+16, 'train_loss': 1.2067382006480176, 'epoch': 2.0})

In [ ]:
# Gửi dữ liệu đào tạo lên Hub
trainer.push_to_hub(commit_message="Training complete")

CommitInfo(commit_url='https://huggingface.co/tienhuynh/distilbert-finetuned-squadv2/commit/54f7249a0f6f454db3a45c1f82413aa9d8692fb6', commit_message='Training complete', commit_description='', oid='54f7249a0f6f454db3a45c1f82413aa9d8692fb6', pr_url=None, pr_revision=None, pr_num=None)

## **6. Evaluate model**

In [ ]:
# Tải metric "squad" từ thư viện evaluate
metric = evaluate.load("squad_v2")

In [ ]:
N_BEST = 20 # Số lượng kết quả tốt nhất được lựa chọn sau khi dự đoán
MAX_ANS_LENGTH = 30 # Độ dài tối đa cho câu trả lời dự đoán

def compute_metrics(start_logits, end_logits, features, examples):
    # Tạo một từ điển mặc định để ánh xạ mỗi ví dụ với danh sách các đặc trưng tương ứng
    example_to_features = collections.defaultdict(list)
    for idx, feature in enumerate(features):
      example_to_features[feature['example_id']].append(idx)

    predicted_answers = []
    for example in tqdm(examples):
      example_id = example['id']
      context = example['context']
      answers = []

      # Lặp qua tất cả các đặc trưng liên quan đến ví dụ đó
      for feature_index in example_to_features[example_id]:
        start_logit = start_logits[feature_index]
        end_logit = end_logits[feature_index]
        offsets = features[feature_index]['offset_mapping']

        # Lấy các chỉ số có giá trị lớn nhất cho start và end logits
        start_indexes = np.argsort(start_logit)[-1 : -N_BEST - 1 : -1].tolist()
        end_indexes = np.argsort(end_logit)[-1 : -N_BEST - 1 : -1].tolist()

        for start_index in start_indexes:
          for end_index in end_indexes:
            # Bỏ qua các câu trả lời không hoàn toàn nằm trong ngữ cảnh
            if offsets[start_index] is None or offsets[end_index] is None:
              continue
            # Bỏ qua các câu trả lời có độ dài > max_answer_length
            if end_index - start_index + 1 > MAX_ANS_LENGTH:
              continue

            # Tạo một câu trả lời mới
            answer = {
                'text': context[offsets[start_index][0] : offsets[end_index][1]],
                'logit_score': start_logit[start_index] + end_logit[end_index],
            }
            answers.append(answer)
      # Chọn câu trả lời có điểm số tốt nhất
      # Chọn câu trả lời có điểm số tốt nhất
      if len(answers) > 0:
          best_answer = max(answers, key=lambda x: x['logit_score'])
          answer_dict = {
            'id': example_id,
            'prediction_text': best_answer['text'],
            'no_answer_probability': 1 - best_answer['logit_score']
          }
      else:
          answer_dict = {
              'id': example_id,
              'prediction_text': '',
              'no_answer_probability': 1.0
          }
      predicted_answers.append(answer_dict)

  # Tạo danh sách câu trả lời lý thuyết từ các ví dụ
    theoretical_answers = [
      {'id': ex['id'], 'answers': ex['answers']} for ex in examples
        ]
    # Sử dụng metric.compute để tính toán các độ đo và trả về kết quả
    return metric.compute(predictions=predicted_answers, references=theoretical_answers)

In [ ]:
# Thực hiện dự đoán trên tập dữ liệu validation
predictions, _, _ = trainer.predict(validation_dataset)

# Lấy ra thông tin về các điểm bắt đầu và điểm kết thúc của câu trả lời dự đoán
start_logits, end_logits = predictions

# Tính toán các chỉ số đánh giá sử dụng hàm compute_metrics
results = compute_metrics(
    start_logits,
    end_logits,
    validation_dataset,
    raw_datasets["validation"]
)
results

  0%|          | 0/11873 [00:00<?, ?it/s]

{'exact': 46.500463235913415,
 'f1': 50.67171378596023,
 'total': 11873,
 'HasAns_exact': 74.59514170040485,
 'HasAns_f1': 82.94960488878291,
 'HasAns_total': 5928,
 'NoAns_exact': 18.486122792262407,
 'NoAns_f1': 18.486122792262407,
 'NoAns_total': 5945,
 'best_exact': 64.92040764760381,
 'best_exact_thresh': -9.90625,
 'best_f1': 66.99835293571564,
 'best_f1_thresh': -9.34375}

## **8. Load model from Hub**

In [ ]:
# Use a pipeline as a high-level helper
from transformers import pipeline

PIPELINE_NAME = 'question-answering'
MODEL_NAME = 'tienhuynh/distilbert-finetuned-squadv2'
pipe = pipeline(PIPELINE_NAME, model=MODEL_NAME)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/561 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


model.safetensors:   0%|          | 0.00/265M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.20k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/712k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

In [ ]:
INPUT_QUESTION = 'What is my name?'
INPUT_CONTEXT = 'My name is Tien and I live in Vietnam.'
pipe(question=INPUT_QUESTION, context=INPUT_CONTEXT)

{'score': 0.9284388422966003, 'start': 11, 'end': 15, 'answer': 'Tien'}